In [5]:
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display
import matplotlib.ticker as ticker
import mplcursors

# Import data, change column header from Sex to Gender
df = pd.read_csv('../Resources/Cleaning2/merged_events_bio.csv')
df = df.rename(columns={'Sex': 'Gender'})
df.head(5)

# Define the medal colors 
color_scale = {'Gold': 'gold', 'Silver': 'silver', 'Bronze': 'peru'}

# Define dropdown menus, alphabetize sports and countries
gender_dropdown = widgets.Dropdown(
    options=['Both', 'M', 'F'],
    value='Both',
    description='Gender:')

sports = ["ALL"] + sorted(df['Sport'].unique().tolist())
sports.sort()
sport_dropdown = widgets.Dropdown(options=sports, value="ALL", description="Sport:")

countries = ["ALL"] + sorted(df['Country_Name'].unique().tolist())
countries.sort()
country_dropdown = widgets.Dropdown(options=countries, value="ALL", description='Country:')

# Define the callback functions
def on_gender_change(change):
    update_scatter_plot(filtered_df, change.new, sport_dropdown.value, country_dropdown.value)
    
def on_sport_change(change):
    update_scatter_plot(filtered_df, gender_dropdown.value, change.new, country_dropdown.value)
    
def on_country_change(change):
    update_scatter_plot(filtered_df, gender_dropdown.value, sport_dropdown.value, change.new)

# Define the function to filter the data 
def filter_data(df, gender, country=None, sport=None):
    filtered_df = df.copy()
    if gender != 'Both':
        filtered_df = filtered_df[filtered_df['Gender'] == gender]
    if country != 'ALL':
        filtered_df = filtered_df[filtered_df['Country_Name'] == country]
    if sport != 'ALL':
        filtered_df = filtered_df[filtered_df['Sport'] == sport]
    
    # filter out rows where no medal was received
    filtered_df = filtered_df[~(filtered_df['Medal'] == 'na')]
        
    return filtered_df

# Add hover labels
def medal_label_formatter(val, pos):
    """Function to format the label of each bar in the medal bar chart."""
    gold_count = int(val[2])
    silver_count = int(val[1])
    bronze_count = int(val[0])
    total_count = gold_count + silver_count + bronze_count
    return f"Gold: {gold_count}\nSilver: {silver_count}\nBronze: {bronze_count}\nTotal: {total_count}"

# Define the function to update the medal bar chart
def update_medal_chart(gender, sport, country):
    filtered_df = filter_data(df, gender, country, sport)
    filtered_df = filtered_df[filtered_df['Age'].notnull()] # Filter out rows where Age is null
    filtered_df['Age'] = filtered_df['Age'].astype(int) # Convert Age column to integer
    medal_counts = pd.pivot_table(filtered_df, index=['Age'], columns=['Medal'], values='Name', aggfunc='count', fill_value=0)
    ax = medal_counts.plot(kind='bar', stacked=True, color=[color_scale.get(x, '#333333') for x in ['Gold', 'Silver', 'Bronze']])
    ax.set_xlabel('Age')
    ax.set_ylabel('Medal Count')
    ax.legend(title='Medal Count by Age', labels=['Gold', 'Silver', 'Bronze'])

    # Add hover labels using mplcursors
    cursor = mplcursors.cursor(ax, hover=True)
    cursor.connect("add", lambda sel: sel.annotation.set_text(medal_label_formatter(sel.target)))

    plt.show()
    
# Define the function to reset the dropdown menus
def reset_dropdowns():
    gender_dropdown.value = 'Both'
    sport_dropdown.value = 'ALL'
    country_dropdown.value = 'ALL'
    
# Register the callback functions
gender_dropdown.observe(on_gender_change, names='value')
sport_dropdown.observe(on_sport_change, names='value')
country_dropdown.observe(on_country_change, names='value')

# Display the dropdown menus
display(gender_dropdown)
display(sport_dropdown)
display(country_dropdown)

# Display the plot
filtered_df = df.copy()
update_medal_chart(gender_dropdown.value, sport_dropdown.value, country_dropdown.value)

ModuleNotFoundError: No module named 'mplcursors'